In [ ]:
!pip install nltk numpy openpyxl Pillow pyparsing scikit-learn scipy keras scipy tensorflow transformers tensorflow-text tf-models-official

     |████████████████████████████████| 2.1MB 29.6MB/s 
     |████████████████████████████████| 3.4MB 36.1MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 
     |████████████████████████████████| 3.3MB 33.4MB/s 
     |████████████████████████████████| 901kB 36.9MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 1.2MB 37.9MB/s 
     |████████████████████████████████| 174kB 57.1MB/s 
     |████████████████████████████████| 358kB 42.4MB/s 
     |████████████████████████████████| 706kB 47.9MB/s 
     |████████████████████████████████| 38.2MB 73kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=db47503492d1bb8c7ae75741b8cd979d31223737c064a3c35a9bf428368acfd5
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any

In [ ]:
import nltk
import re
from openpyxl import load_workbook
from sklearn import preprocessing
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertMainLayer, TFBertPreTrainedModel
import os
import shutil
import matplotlib.pyplot as plt
tf.get_logger().setLevel("ERROR")

In [ ]:
ALL_CLASSES = [
    "content discussion", "greeting", "logistics", "assignment instructions", "instruction question",
    "assignment question", "general comment", "response", "incomplete/typo", "feedback",
    "emoticon/non-verbal", "discussion wrap-up", "outside material", "opening statement",
    "general question", "content question", "general discussion"
]
DATASET_PATH = "dataset.xlsx"

# Opens a single sheet for reading, returns a list of messages and a list of classes
def read_worksheet(filename, sheet_name, all_classes, label_encoder, no_columns):
    wb = load_workbook(filename, read_only=True)
    ws = wb[sheet_name]
    column_labels = next(ws.rows)
    X = []
    y = []
    for row in ws.rows:
        if row[0].value is None:
            break
        elif row[0].value.strip() == "Course":  # Skip the first line which only contains column titles
            continue
        new_entry = {}
        for i in range(no_columns):
            new_entry[column_labels[i].value.lower().replace(" ", "_")] = str(row[i].value)
        c_list = [new_entry["codepreliminary"].lower().strip()]
        if c_list[0] not in all_classes:
            c_list = new_entry["codepreliminary"].lower().strip().split("/")
        # If there are 2 classes listed in document add message twice (1 for each class)
        for c in c_list:
            new_entry["codepreliminary"] = label_encoder.transform([c])[0]
            X.append(new_entry["message"])
            y.append(label_encoder.transform([c])[0])
    wb.close()
    return X, y

le = preprocessing.LabelEncoder()
le.fit(ALL_CLASSES)

LabelEncoder()

In [ ]:
X, y = read_worksheet(DATASET_PATH, "Discussion only data", ALL_CLASSES, le, 10)
X2, y2 = read_worksheet(DATASET_PATH, "CREW data", ALL_CLASSES, le, 11)
X.extend(X2)  # Join worksheets into a single dataset
y.extend(y2)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length = 256
batch_size = 6

In [ ]:
def map_data_to_dict(input_ids, attention_masks, token_type_ids, classes):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, classes

In [ ]:
def encode_data(data, classes):
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  class_list = []
  for i, msg in enumerate(data):
    input = tokenizer.encode_plus(
              msg,
              add_special_tokens=True,
              max_length=max_length,
              pad_to_max_length=True,
              return_attention_mask=True
            )
    input_ids_list.append(input["input_ids"])
    token_type_ids_list.append(input["token_type_ids"])
    attention_mask_list.append(input["attention_mask"])
    class_list.append([classes[i]])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, class_list)).map(map_data_to_dict)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = encode_data(X_train, y_train).shuffle(5000).batch(batch_size)
test_dataset = encode_data(X_test, y_test).batch(batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
class ModifiedBertModel(TFBertPreTrainedModel):
    def __init__(self, config,
                 no_classes,
                 dropout_rate=0.1,
                 cnn_filters=50,
                 training=False,
                 *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert", trainable = True)
        
        self.cnn_layer1 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = tf.keras.layers.GlobalMaxPool1D()
        self.dense_1 = tf.keras.layers.Dense(units=512, activation="relu")
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        self.last_dense = tf.keras.layers.Dense(units=no_classes, activation="softmax")

    def call(self, inputs, training = True, **kwargs):        
        bert_outputs = self.bert(inputs, training = training, **kwargs)

        l_1 = self.cnn_layer1(bert_outputs[0]) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(bert_outputs[0]) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(bert_outputs[0])
        l_3 = self.pool(l_3) 

        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        return model_output

In [ ]:
def build_model(learning_rate=3e-5, no_epochs=5):
  model = ModifiedBertModel.from_pretrained("bert-base-uncased", no_classes=len(ALL_CLASSES), dropout_rate=0.3)
  #TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(ALL_CLASSES))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
  return model

In [ ]:
def get_results(test_data, model, print_output=True):
  results_true = test_data.unbatch()
  results_true = np.asarray([element[1].numpy() for element in results_true])
  results = model.predict(test_data)
  results_predicted = np.argmax(results, axis=1)

  accuracy = accuracy_score(results_true, results_predicted)
  precision = precision_score(results_true, results_predicted, zero_division=1, average="weighted")
  recall = recall_score(results_true, results_predicted, zero_division=1, average="weighted")
  f1 = f1_score(results_true, results_predicted, zero_division=1, average="weighted")

  if print_output:
    print("Accuracy score: {}".format(accuracy))
    print("Precision score: {}".format(precision))
    print("Recall score: {}".format(recall))
    print("F1 score: {}".format(f1))

  return accuracy, precision, recall, f1

In [ ]:
model = build_model()
history = model.fit(train_dataset, epochs=no_epochs, validation_data=test_dataset)
get_results(test_dataset, model)

Epoch 1/5
113/113 [==============================] - 60s 394ms/step - loss: 2.1604 - accuracy: 0.3824 - val_loss: 1.3970 - val_accuracy: 0.5976
Epoch 2/5
113/113 [==============================] - 44s 387ms/step - loss: 0.9827 - accuracy: 0.7303 - val_loss: 1.2520 - val_accuracy: 0.6391
Epoch 3/5
113/113 [==============================] - 43s 385ms/step - loss: 0.5287 - accuracy: 0.8574 - val_loss: 1.2653 - val_accuracy: 0.6686
Epoch 4/5
113/113 [==============================] - 44s 388ms/step - loss: 0.3554 - accuracy: 0.8944 - val_loss: 1.2716 - val_accuracy: 0.7041
Epoch 5/5
113/113 [==============================] - 44s 387ms/step - loss: 0.2015 - accuracy: 0.9409 - val_loss: 1.1940 - val_accuracy: 0.6864


In [ ]:
no_folds = 5
kfold = KFold(no_folds, shuffle=True)

# Convert to numpy arrays so indexing with list of indexes works
X = np.array(X) 
y = np.array(y)

test_accuracy_log = []

accuracy = []
precision = []
recall = []
f1 = []
for train, test in kfold.split(X, y):
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]

  train_dataset = encode_data(X_train, y_train).shuffle(5000).batch(batch_size)
  test_dataset = encode_data(X_test, y_test).batch(batch_size)
  
  model = build_model()
  model.fit(train_dataset, epochs=no_epochs, validation_data=test_dataset, verbose=1)

  acc, pre, rec, f = get_results(test_dataset, model, print_output=False)
  accuracy.append(acc)
  precision.append(pre)
  recall.append(rec)
  f1.append(f)

print("Accuracy score: {}".format(np.mean(accuracy)))
print("Precision score: {}".format(np.mean(precision)))
print("Recall score: {}".format(np.mean(recall)))
print("F1 score: {}".format(np.mean(f1)))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some layers from the model checkpoint at bert-base-uncased were not used when initializing ModifiedBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing ModifiedBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ModifiedBertModel from the checkpoint of a model that

Epoch 1/5
113/113 [==============================] - 61s 412ms/step - loss: 2.0028 - accuracy: 0.4472 - val_loss: 1.4514 - val_accuracy: 0.6036
Epoch 2/5
113/113 [==============================] - 43s 385ms/step - loss: 1.0622 - accuracy: 0.6972 - val_loss: 1.2579 - val_accuracy: 0.6568
Epoch 3/5
113/113 [==============================] - 44s 389ms/step - loss: 0.6063 - accuracy: 0.8267 - val_loss: 1.1706 - val_accuracy: 0.6686
Epoch 4/5
113/113 [==============================] - 44s 389ms/step - loss: 0.2956 - accuracy: 0.9207 - val_loss: 1.1672 - val_accuracy: 0.6746
Epoch 5/5
113/113 [==============================] - 44s 388ms/step - loss: 0.1599 - accuracy: 0.9589 - val_loss: 1.2306 - val_accuracy: 0.6864


Some layers from the model checkpoint at bert-base-uncased were not used when initializing ModifiedBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing ModifiedBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ModifiedBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of ModifiedBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dense_48', 'dropout_1138', 'global_max_pooling1d_23', 'conv1d_67', 'conv1d_68', 'dense_47', 'conv1d_66']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 63s 421ms/step - loss: 2.0857 - accuracy: 0.4611 - val_loss: 1.4358 - val_accuracy: 0.5799
Epoch 2/5
113/113 [==============================] - 44s 387ms/step - loss: 1.1229 - accuracy: 0.6540 - val_loss: 1.1403 - val_accuracy: 0.6568
Epoch 3/5
113/113 [==============================] - 44s 390ms/step - loss: 0.6148 - accuracy: 0.8412 - val_loss: 1.0695 - val_accuracy: 0.6805
Epoch 4/5
113/113 [==============================] - 44s 389ms/step - loss: 0.2841 - accuracy: 0.9156 - val_loss: 1.1059 - val_accuracy: 0.6627
Epoch 5/5
113/113 [==============================] - 44s 388ms/step - loss: 0.2138 - accuracy: 0.9489 - val_loss: 1.3190 - val_accuracy: 0.6805


Some layers from the model checkpoint at bert-base-uncased were not used when initializing ModifiedBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing ModifiedBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ModifiedBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of ModifiedBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['conv1d_70', 'conv1d_71', 'global_max_pooling1d_24', 'dense_49', 'dense_50', 'conv1d_69', 'dropout_1176']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 63s 414ms/step - loss: 2.0494 - accuracy: 0.4435 - val_loss: 1.2656 - val_accuracy: 0.6391
Epoch 2/5
113/113 [==============================] - 44s 386ms/step - loss: 1.0849 - accuracy: 0.6845 - val_loss: 1.0244 - val_accuracy: 0.7041
Epoch 3/5
113/113 [==============================] - 44s 389ms/step - loss: 0.5735 - accuracy: 0.8323 - val_loss: 0.8132 - val_accuracy: 0.7929
Epoch 4/5
113/113 [==============================] - 44s 387ms/step - loss: 0.3008 - accuracy: 0.9329 - val_loss: 0.8187 - val_accuracy: 0.7988
Epoch 5/5
113/113 [==============================] - 44s 389ms/step - loss: 0.1591 - accuracy: 0.9642 - val_loss: 0.8351 - val_accuracy: 0.7751


Some layers from the model checkpoint at bert-base-uncased were not used when initializing ModifiedBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing ModifiedBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ModifiedBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of ModifiedBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['conv1d_72', 'global_max_pooling1d_25', 'dropout_1214', 'dense_52', 'conv1d_73', 'dense_51', 'conv1d_74']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 62s 412ms/step - loss: 1.9593 - accuracy: 0.5036 - val_loss: 1.3860 - val_accuracy: 0.5833
Epoch 2/5
113/113 [==============================] - 44s 387ms/step - loss: 1.2456 - accuracy: 0.6286 - val_loss: 0.9835 - val_accuracy: 0.7500
Epoch 3/5
113/113 [==============================] - 44s 390ms/step - loss: 0.7194 - accuracy: 0.8048 - val_loss: 0.8116 - val_accuracy: 0.7857
Epoch 4/5
113/113 [==============================] - 44s 389ms/step - loss: 0.3388 - accuracy: 0.9427 - val_loss: 0.8352 - val_accuracy: 0.7798
Epoch 5/5
113/113 [==============================] - 44s 389ms/step - loss: 0.1602 - accuracy: 0.9618 - val_loss: 0.8777 - val_accuracy: 0.7857


Some layers from the model checkpoint at bert-base-uncased were not used when initializing ModifiedBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing ModifiedBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ModifiedBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of ModifiedBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dense_54', 'conv1d_75', 'conv1d_77', 'dropout_1252', 'dense_53', 'conv1d_76', 'global_max_pooling1d_26']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
113/113 [==============================] - 61s 410ms/step - loss: 2.2150 - accuracy: 0.3720 - val_loss: 1.2529 - val_accuracy: 0.6190
Epoch 2/5
113/113 [==============================] - 43s 384ms/step - loss: 1.1036 - accuracy: 0.7052 - val_loss: 0.9981 - val_accuracy: 0.7083
Epoch 3/5
113/113 [==============================] - 44s 387ms/step - loss: 0.6904 - accuracy: 0.8131 - val_loss: 0.8843 - val_accuracy: 0.7619
Epoch 4/5
113/113 [==============================] - 44s 387ms/step - loss: 0.4035 - accuracy: 0.8921 - val_loss: 0.8964 - val_accuracy: 0.7440
Epoch 5/5
113/113 [==============================] - 44s 388ms/step - loss: 0.2086 - accuracy: 0.9541 - val_loss: 0.8523 - val_accuracy: 0.7798
Accuracy score: 0.7414976049591433
Precision score: 0.7577498840662374
Recall score: 0.7414976049591433
F1 score: 0.7226733420388161
